# Baseline for datasets
Insert train.json, test.json under the folder data to get data

In [1]:
from CNN.config import Config
from CNN.utils import WordEmbeddingLoader, RelationLoader, SemEvalDataLoader
import numpy as np
config = Config()
config.batch_size = 1
config.embedding_path = "./CNN/embedding/hlbl-embeddings-scaled.EMBEDDING_SIZE=50.txt"
config.data_dir = "./CNN/data/simeval2007/"
word2id, word_vec = WordEmbeddingLoader(config).load_embedding()
rel2id, id2rel, class_num = RelationLoader(config).get_relation()
loader = SemEvalDataLoader(rel2id, word2id, config)

/home/dmitrii/anaconda3/envs/psdaub4/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Get X and y vectors from dataset
NB: X is already a feature vector got from the word embedding

In [2]:
def get_x_y_from_loader(loader):
    # upload train and test from dataloader
    X = []
    y = []
    for step, (data, label) in enumerate(loader):
        x = data.detach().numpy().flatten()
        x.astype(int)
        X.append(x)
        y.append(label.detach().numpy()[0])
    X = np.array(X)
    y = np.array(y)
    return X, y

X_train, y_train = get_x_y_from_loader(loader.get_train())
X_test, y_test = get_x_y_from_loader(loader.get_test())

## Metrics
According to the [survey](https://link.springer.com/content/pdf/10.1007/s10115-022-01665-w.pdf), the main metrics are Accuracy, Precision, Recall and F1 Score. However, they may generate overoptimistic, misleading results on
imbalanced datasets, as they failed to consider the ratio between positive and negative classes. To mitigate this, Matthews Correlation Coefficient (It is high only when the classifier is doing well in both positive and negative
classes) and G-Mean-Score (a poor performance in positive examples prediction will lead to a low G-mean
value, even if negative instances are correctly classified by the classifier) are used.

In [9]:
from custom_statistics import Statistics
stats = Statistics()
random_states = [0, 1, 42, 100, 5782]

## Baseline: Bayessian Classifier
According to the [survey](https://link.springer.com/content/pdf/10.1007/s10115-022-01665-w.pdf), different baselines are possible. For instance, Logistic Regression, Decision Trees, SVM. [Sorgente et al.](http://ceur-ws.org/Vol-1109/paper4.pdf) used Naive Bayes.

### Naive Bayes

In [4]:
from sklearn.naive_bayes import MultinomialNB
nb = MultinomialNB()
nb.fit(X_train, y_train)
predictions_nb = nb.predict(X_test)
stats.get_metrics(predictions_nb, y_test, show=True)

Accuracy Score ->  0.6721311475409836
Precision Score ->  0.8771845575408872
Recall Score ->  0.6721311475409836
F1 Score ->  0.7538814854748006
Matthews Correlation Coefficient ->  0.069449619819268
G Mean Score ->  0.0


/home/dmitrii/anaconda3/envs/psdaub4/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/dmitrii/anaconda3/envs/psdaub4/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


(0.6721311475409836,
 0.8771845575408872,
 0.6721311475409836,
 0.7538814854748006,
 0.069449619819268,
 0.0)

## Other possible baselines
### Logistic Regression

In [5]:
stats.clean()
from sklearn.linear_model import LogisticRegression
for random_state in random_states:
    logreg = LogisticRegression(solver='lbfgs', random_state=random_state, multi_class='multinomial', max_iter=100, n_jobs=16)
    logreg.fit(X_train, y_train)
    predictions_logreg = logreg.predict(X_test)
    stats.add(predictions_logreg, y_test)
stats.show()

/home/dmitrii/anaconda3/envs/psdaub4/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/home/dmitrii/anaconda3/envs/psdaub4/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/l

Accuracy Score ->  (0.9034608378870674, 0.0)
Precision Score ->  (0.8546705877963319, 0.0)
Recall Score ->  (0.9034608378870674, 0.0)
F1 Score ->  (0.8783887189409192, 0.0)
Matthews Correlation Coefficient ->  (-0.042468170510776115, 0.0)
G Mean Score ->  (0.0, 0.0)


/home/dmitrii/anaconda3/envs/psdaub4/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


### SVM
NB: for large datasets can have a long training period

In [6]:
from sklearn import svm
stats.clean()
for random_state in random_states:
    SVM = svm.SVC(random_state=random_state)
    SVM.fit(X_train, y_train)
    predictions_SVM = SVM.predict(X_test)
    stats.add(predictions_SVM, y_test)
stats.show()

Accuracy Score ->  (0.9253187613843352, 1.1102230246251565e-16)
Precision Score ->  (0.8562148101698401, 0.0)
Recall Score ->  (0.9253187613843352, 1.1102230246251565e-16)
F1 Score ->  (0.8894265483126628, 0.0)
Matthews Correlation Coefficient ->  (0.0, 0.0)
G Mean Score ->  (0.0, 0.0)


/home/dmitrii/anaconda3/envs/psdaub4/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/dmitrii/anaconda3/envs/psdaub4/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/dmitrii/anaconda3/envs/psdaub4/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/dmitrii/anaconda3/e

### Decision Tree

In [7]:
from sklearn.tree import DecisionTreeClassifier
stats.clean()
for random_state in random_states:
    rfc = DecisionTreeClassifier(random_state=random_state)
    rfc.fit(X_train, y_train)
    predictions_rfc = rfc.predict(X_test)
    stats.add(predictions_rfc, y_test)
stats.show()

Accuracy Score ->  (0.8553734061930782, 0.008590037247760718)
Precision Score ->  (0.863580101846375, 0.0035151435963382788)
Recall Score ->  (0.8553734061930782, 0.008590037247760718)
F1 Score ->  (0.8594367208110436, 0.006057294967539929)
Matthews Correlation Coefficient ->  (0.01267224942965979, 0.024570110010172615)
G Mean Score ->  (0.0, 0.0)


/home/dmitrii/anaconda3/envs/psdaub4/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/dmitrii/anaconda3/envs/psdaub4/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/dmitrii/anaconda3/envs/psdaub4/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/dmitrii/anaconda3/envs/psdaub4/lib/python3.

### Random Forest

In [8]:
from sklearn.ensemble import RandomForestClassifier
stats.clean()
for random_state in random_states:
    rfc = RandomForestClassifier(random_state=random_state)
    rfc.fit(X_train, y_train)
    predictions_rfc = rfc.predict(X_test)
    stats.add(predictions_rfc, y_test)
stats.show()

Accuracy Score ->  (0.926047358834244, 0.0008923459900849656)
Precision Score ->  (0.8867122799300986, 0.037351619679296684)
Recall Score ->  (0.926047358834244, 0.0008923459900849656)
F1 Score ->  (0.8911859520913913, 0.002154820754704681)
Matthews Correlation Coefficient ->  (0.06014640022949119, 0.07366399521373523)
G Mean Score ->  (0.06246950475544242, 0.07650920556760059)


/home/dmitrii/anaconda3/envs/psdaub4/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/dmitrii/anaconda3/envs/psdaub4/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/dmitrii/anaconda3/envs/psdaub4/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
